# Import der benötigten Bibliotheken
Importiere die notwendigen Bibliotheken, einschließlich Semantic Kernel.

In [1]:
# Import der benötigten Bibliotheken

import os
import sys
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import KernelArguments

# Kernel einrichten
Richte den Kernel ein und verbinde ihn mit dem LLM-Dienst.

In [2]:
# Kernel einrichten

# Initialisiere den Kernel
kernel = Kernel()

# Verbinde den Kernel mit dem LLM-Dienst
service_id = "philosopher-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

settings = chat_service.get_prompt_execution_settings_class()
settings.temperature = 0.7
settings.top_p = 0.8

kernel.add_service(chat_service)

# Teste den Kernel
print("Kernel erfolgreich initialisiert!")

Kernel erfolgreich initialisiert!


# Philosophische Texte generieren
Erstelle eine Funktion, die philosophische Texte generiert.

In [3]:
# Funktion zum Generieren philosophischer Texte

system_message = """
Du bist ein weiser Philosoph. Schreibe einen tiefgründigen und nachdenklichen Text über das Thema: {{$topic}}.
"""

philosophical_function = kernel.add_function(
    prompt=system_message + """{{$chat_history}}{{$user_input}}""",
    function_name="GeneratePhilosophicalText",
    plugin_name="Philosopher",
    settings=settings,
)

history = ChatHistory()
history.add_user_message("Hi there, who are you?")
history.add_assistant_message("I am Philosophus. I write perfect Philosophical Texts.")

# Beispielaufruf der Funktion
topic = "Die Natur der Realität"
input = f"Schreibe einen Text über {topic}."
response = await kernel.invoke(philosophical_function, user_input=input, chat_history=history)

print(response)
history.add_user_message(input)
history.add_assistant_message(str(response))

Variable `Symbols.VAR_PREFIX: topic` not found in the KernelArguments


Die Natur der Realität ist ein Thema, das Philosophen, Wissenschaftler und Denker seit Jahrtausenden beschäftigt. Diese Frage führt uns tief in das Wesen des Seins, in die Strukturen der Wahrnehmung und in die unendlichen Dimensionen des Denkens. Was ist tatsächlich real? Ist es die materielle Welt, die wir mit unseren Sinnen erfassen können, oder gibt es eine tiefere, metaphysische Realität, die unseren Verstand übersteigt?

Wir leben in einer Welt, die uns durch unsere Sinne zugänglich gemacht wird. Unser Alltag ist geprägt von den sichtbaren Formen, den hörbaren Klängen und den fühlbaren Texturen. Doch die Philosophie verlangt von uns, über das Offensichtliche hinauszudenken. Plato stellte in seiner Höhlengleichnis dar, dass das, was wir als Realität wahrnehmen, lediglich Schatten von einer höheren Form der Wahrheit sind. Die Realität, so argumentiert er, ist das, was hinter den Schatten liegt – eine Welt der Ideen, die über unsere sensorische Erfahrung hinausgeht.

Neuere philosoph

# Text in Meme-Text umwandeln
Erstelle eine AI-gestützte Funktion, die den Text in Meme-Text umwandelt.

In [4]:
meme_settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
meme_settings.temperature = 0.8

meme_prompt_template = """
Input: {{$input}}
Task: Verwandle den folgenden philosophischen Text in einen lustigen und ansprechenden Meme-Text.
Philosophischer Text: {{$philosophical_text}}

Output your response in this format:
1. [Meme-Text]
"""

meme_function = kernel.add_function(
    plugin_name="MemeGenerator",
    function_name="GenerateMemeText",
    prompt=meme_prompt_template,
    template_format="semantic-kernel",
    prompt_execution_settings=meme_settings,
)

philosophical_text = "Die Natur der Realität ist eine tiefgründige und komplexe Frage, die seit Jahrhunderten Philosophen beschäftigt."
input = f"Verwandle diesen Text in einen Meme-Text: {philosophical_text}"

result = await kernel.invoke(
    meme_function,
    KernelArguments(input=input, philosophical_text=philosophical_text)
)

print(result)

1. "Reality: It's like a never-ending riddle that has been confusing philosophers since forever. 🤔💭"


# Text in Emojis umwandeln
Erstelle eine AI-gestützte Funktion, die den Text in Emojis umwandelt.

In [5]:
emoji_settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
emoji_settings.temperature = 0.8

emoji_prompt_template = """
Input: {{$input}}
Task: Verwandle den folgenden philosophischen Text in eine Emoji-Darstellung.
Philosophischer Text: {{$philosophical_text}}

Output your response in this format:
1. [Emoji-Text]
"""

emoji_function = kernel.add_function(
    plugin_name="EmojiConverter",
    function_name="ConvertToEmoji",
    prompt=emoji_prompt_template,
    template_format="semantic-kernel",
    prompt_execution_settings=emoji_settings,
)

philosophical_text = "Die Natur der Realität ist eine tiefgründige und komplexe Frage, die seit Jahrhunderten Philosophen beschäftigt."
input = f"Verwandle diesen Text in eine Emoji-Darstellung: {philosophical_text}"

result = await kernel.invoke(
    emoji_function,
    KernelArguments(input=input, philosophical_text=philosophical_text)
)

print(result)

1. 🌿🌍🤔🔍💭❓🕰️👨‍🎓👩‍🎓💡📜


Nicht KI Textanalyse die Anzahl Wörter und Zeichen zählt

In [6]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import KernelArguments, KernelFunction

class TextAnalysisPlugin:
    """Description: TextAnalysisPlugin provides a set of functions to analyze text.

    Usage:
        kernel.add_plugin(TextAnalysisPlugin(), plugin_name="text_analysis")

    Examples:
        {{text_analysis.WordCount}} => Returns the number of words in the input text
        {{text_analysis.CharacterCount}} => Returns the number of characters in the input text
    """

    @kernel_function(name="WordCount")
    def word_count(
        self, input: Annotated[str, "The text to analyze"]
    ) -> Annotated[int, "The number of words in the text"]:
        """Returns the number of words in the provided text."""
        return len(input.split())

    @kernel_function(name="CharacterCount")
    def character_count(
        self, input: Annotated[str, "The text to analyze"]
    ) -> Annotated[int, "The number of characters in the text"]:
        """Returns the number of characters in the provided text."""
        return len(input)

# Register the plugin with the kernel
text_analysis_plugin = kernel.add_plugin(TextAnalysisPlugin(), "text_analysis")

philosophical_text = "Die Natur der Realität ist eine tiefgründige und komplexe Frage, die seit Jahrhunderten Philosophen beschäftigt."

# Beispielaufruf der Funktionen
word_count_function = kernel.get_function("text_analysis", "WordCount")
word_count_result = await kernel.invoke(
    word_count_function,
    KernelArguments(input=philosophical_text)
)
print(f"Word Count: {word_count_result}")

character_count_function = kernel.get_function("text_analysis", "CharacterCount")
character_count_result = await kernel.invoke(
    character_count_function,
    KernelArguments(input=philosophical_text)
)
print(f"Character Count: {character_count_result}")

Word Count: 15
Character Count: 112


Kombination beider KI Funktionen

In [7]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import KernelArguments

class TextTransformationPlugin:
    """Description: TextTransformationPlugin provides functions to transform text into memes and emojis.

    Usage:
        kernel.add_plugin(TextTransformationPlugin(), plugin_name="text_transformation")
    """

    @kernel_function(name="GenerateMemeText")
    async def generate_meme_text(self, input: Annotated[str, "The text to transform into a meme"]) -> str:
        """Transforms the provided text into a meme."""
        meme_prompt_template = """
        Input: {{$input}}
        Task: Verwandle den folgenden philosophischen Text in einen lustigen und ansprechenden Meme-Text.
        Philosophischer Text: {{$philosophical_text}}

        Output your response in this format:
        1. [Meme-Text]
        """
        
        meme_function = kernel.add_function(
            plugin_name="MemeGenerator",
            function_name="GenerateMemeText",
            prompt=meme_prompt_template,
            template_format="semantic-kernel",
            prompt_execution_settings=meme_settings,
        )

        return await kernel.invoke(meme_function, KernelArguments(input=input, philosophical_text=input))

    @kernel_function(name="ConvertToEmoji")
    async def convert_to_emoji(self, input: Annotated[str, "The text to transform into emojis"]) -> str:
        """Transforms the provided text into emojis."""
        emoji_prompt_template = """
        Input: {{$input}}
        Task: Verwandle den folgenden philosophischen Text in eine Emoji-Darstellung.
        Philosophischer Text: {{$philosophical_text}}

        Output your response in this format:
        1. [Emoji-Text]
        """
        
        emoji_function = kernel.add_function(
            plugin_name="EmojiConverter",
            function_name="ConvertToEmoji",
            prompt=emoji_prompt_template,
            template_format="semantic-kernel",
            prompt_execution_settings=emoji_settings,
        )

        return await kernel.invoke(emoji_function, KernelArguments(input=input, philosophical_text=input))

# Register the plugin with the kernel
text_transformation_plugin = kernel.add_plugin(TextTransformationPlugin(), "text_transformation")

philosophical_text = "Die Natur der Realität ist eine tiefgründige und komplexe Frage, die seit Jahrhunderten Philosophen beschäftigt."

print("🔍 Text Transformation Demo\n")
print("📝 Transforming text:", philosophical_text, "\n")

print("1️⃣ Meme Text:")
meme_text = await kernel.invoke(text_transformation_plugin["GenerateMemeText"], KernelArguments(input=philosophical_text))
print(meme_text, "\n")

print("2️⃣ Emoji Text:")
emoji_text = await kernel.invoke(text_transformation_plugin["ConvertToEmoji"], KernelArguments(input=philosophical_text))
print(emoji_text)

🔍 Text Transformation Demo

📝 Transforming text: Die Natur der Realität ist eine tiefgründige und komplexe Frage, die seit Jahrhunderten Philosophen beschäftigt. 

1️⃣ Meme Text:
1. "Wenn du versuchst, die Natur der Realität zu verstehen, aber dein Kaffeebecher schon wieder leer ist: 🤔☕️ 'Philosophie? Ich dachte, wir reden über Snacks!' 🍕😂" 

2️⃣ Emoji Text:
1. 🌍🤔🔍💭❓🧠🕰️👨‍🎓👩‍🎓


Memory

In [8]:
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

# Speicher einrichten
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

# Informationen speichern
await memory.save_information(
    collection="facts", id="fact1", text=str(result), description="Meme Text"
)

await memory.save_information(
    collection="facts", id="fact2", text=str(result), description="Emoji Text"
)

# Fragen definieren
questions = [
    "What was the result of the meme text transformation?",
    "What was the result of the emoji text transformation?",
]

# Fragen stellen und Antworten abrufen
for question in questions:
    print(f"Question: {question}")
    result = await memory.search("facts", question)
    print(f"Answer: {result[0].text}\n")

Question: What was the result of the meme text transformation?
Answer: 1. 🌿🌍🤔🔍💭❓🕰️👨‍🎓👩‍🎓💡📜

Question: What was the result of the emoji text transformation?
Answer: 1. 🌿🌍🤔🔍💭❓🕰️👨‍🎓👩‍🎓💡📜

